In [ ]:
#combination driving building off quaternaryModelDriving file, 
import carla
import pygame
import time
import random
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from carla import VehicleLightState

#steering classes
CLASS_NAMES = [
    "Left Hardest", "Left Harder", "Left Hard", "Left Medium", "Left Light", "Left Slight", "Left Minimal",
    "No Turning",
    "Right Minimal", "Right Slight", "Right Light", "Right Medium", "Right Hard", "Right Harder", "Right Hardest"
]

# PyTorch Model Definition for steering and velocity
class SteeringClassifier(nn.Module):
    def __init__(self, num_classes=15):
        super(SteeringClassifier, self).__init__()
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.cnn = resnet
        self.turn_embed = nn.Embedding(3, 16)
        self.fc = nn.Sequential(
            nn.Linear(512 + 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, turn_signal):
        x_img = self.cnn(image)
        x_signal = self.turn_embed(turn_signal)
        x = torch.cat((x_img, x_signal), dim=1)
        return self.fc(x)
    
#added velocity regression:
class VelocityRegression(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(pretrained=True)
        base.fc = nn.Linear(base.fc.in_features, 1)
        self.model = base

    def forward(self, image):
        return self.model(image)

# Initialize PyTorch Model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SteeringClassifier(num_classes=15)
model.load_state_dict(torch.load("../Models/30.01_quaternary_model_final.pth", map_location=device))
model.eval().to(device)

#intiallizign velocity model
velocity_model = VelocityRegression().to(device)
velocity_model.load_state_dict(torch.load("../Models/velocity_regressor_split.pth", map_location=device))
velocity_model.eval()

image_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# === Pygame Setup ===
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("CARLA Model-Controlled Driving + Velocity Prediction")

# === Connect to CARLA ===
client = carla.Client("localhost", 2000)
client.set_timeout(25.0)
client.load_world("Town05")
world = client.get_world()
blueprint_library = world.get_blueprint_library()

#code to add traffic into the simulation, but turned off right now
# tm = client.get_trafficmanager()
# tm_port = tm.get_port()
# tm.set_synchronous_mode(True)

# background_vehicles = []
# for _ in range(100):
#     npc_bp = random.choice(blueprint_library.filter('vehicle.*'))
#     npc_spawn = random.choice(world.get_map().get_spawn_points())
#     npc = world.try_spawn_actor(npc_bp, npc_spawn)
#     if npc:
#         npc.set_autopilot(True, tm_port)
#         tm.vehicle_percentage_speed_difference(npc, random.randint(0, 30))
#         background_vehicles.append(npc)


# === Spawn Vehicle ===
vehicle_bp = blueprint_library.filter("vehicle.mini.cooper_s_2021")[0]
spawn_point = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
vehicle.set_autopilot(False)

# === Spectator Setup ===
spectator = world.get_spectator()

# === Attach Camera Sensor ===
cam_bp = blueprint_library.find('sensor.camera.rgb')
cam_bp.set_attribute('image_size_x', '448')
cam_bp.set_attribute('image_size_y', '252')
cam_bp.set_attribute('fov', '145')
cam_bp.set_attribute('sensor_tick', '0.1')
cam_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(cam_bp, cam_transform, attach_to=vehicle)

# Shared variables
camera_image = None
camera_np = None
pred_label = "Loading..."
pred_speed_label = "Pred Speed: -- m/s"

def process_image(image):
    global camera_image, camera_np
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]  # RGB
    camera_np = array.copy()
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    camera_image = pygame.transform.scale(camera_image_raw, (800, 600))

camera.listen(process_image)

# === Clock ===
clock = pygame.time.Clock()

print("Model-controlled driving started.")
print("Q/E to turn on left/right signal, R to cancel.")
print("ESC or close window to exit.")

# === Steering Mapping for 15 Classes ===
steering_map = {
    0: -0.8,   1: -0.6,   2: -0.4,   3: -0.25,   4: -0.20,  5: -0.1,  6: -0.05,
    7:  0.0,
    8:  0.05,  9:  0.1, 10:  0.15, 11:  0.25,  12:  0.4,  13:  0.6,  14:  0.8
}

#velocity helper functions functions
def get_speed(actor: carla.Actor) -> float:
    v=actor.get_velocity()
    return float((v.x**2 + v.y**2 + v.z**2)**0.5)

def clamp(x, lo, hi):#clamp function to limit values
    return lo if x < lo else hi if x > hi else x

#PID tuning used to handle speed control
PROPORTIONAL_GAIN = 0.7 #reacts to current speed error
INTEGRAL_GAIN     = 0.02 #corrects small persistent offset
DERIVATIVE_GAIN   = 0.28 #damp oscillation

#max throttle and brake values
INTEGRAL_CLAMP = 5.0
MAX_ACCEL_THROTTLE = 1.0
MAX_BRAKE = 1.0

#break behavoir tweaks/modifiers
#modify these settings to ensure car behaves as desired
BRAKE_GAIN_MULT   = 2.2 #make braking stronger than acceleration equivalent
MIN_BRAKE_TO_BITE = 0.15 # min brake values in CARLA
IGNORE_ERROR      = 0.03 # ignore errors in range

# PID state variables
integral_err = 0.0
prev_err = 0.0 #track previous error
prev_time = time.time() #track previous time

# === Main Loop ===
try:
    while True:
        clock.tick(60)
        pygame.event.pump()
        keys = pygame.key.get_pressed()

        # Turn signal input (Q/E/R)
        if keys[pygame.K_q]:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            turn_signal = -1
        elif keys[pygame.K_e]:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            turn_signal = 1
        elif keys[pygame.K_r]:
            vehicle.set_light_state(VehicleLightState.NONE)
            turn_signal = 0
        else:
            light_state = vehicle.get_light_state()
            if light_state == VehicleLightState.LeftBlinker:
                turn_signal = -1
            elif light_state == VehicleLightState.RightBlinker:
                turn_signal = 1
            else:
                turn_signal = 0

        # === Predict and Control ===
        control = carla.VehicleControl()

        target_speed = None
        predicted_speed = None

        prev_steering_value = 0.0
        last_steering_time = time.time()
        STEERING_HOLD_S = 0.05  # 50 ms

        if camera_np is not None:
            with torch.no_grad():
                img = image_transform(camera_np).unsqueeze(0).to(device)

                #steering prediction
                signal = torch.tensor([[turn_signal + 1]], dtype=torch.long).to(device)
                output = model(img, signal.squeeze(1))
                pred_class = output.argmax(dim=1).item()
                pred_label = CLASS_NAMES[pred_class]
                steering_value = steering_map[pred_class]

                # Enforce minimum turn for active signal
                if turn_signal == -1 and steering_value > -0.02:
                    steering_value = -0.02
                elif turn_signal == 1 and steering_value < 0.02:
                    steering_value = 0.02

                # === Steering control logic ===
                current_time = time.time()

                if steering_value == 0.0 or prev_steering_value == 0.0:
                    # Direct application with no hold or smoothing
                    control.steer = steering_value
                    prev_steering_value = steering_value
                    last_steering_time = current_time
                elif steering_value != prev_steering_value:
                    # New value → average and apply
                    smoothed_value = (steering_value + prev_steering_value) / 2
                    control.steer = smoothed_value
                    prev_steering_value = steering_value
                    last_steering_time = current_time
                else:
                    # Same value → wait until 10ms elapsed
                    if current_time - last_steering_time >= STEERING_HOLD_S:
                        control.steer = steering_value

                #velocity prediction
                velocity_out = velocity_model(img)
                raw_speed = (float(velocity_out.squeeze().item()))
                raw_speed = max(0.0, raw_speed)#so the car doesn't go in negative speed

                predicted_speed = raw_speed
                pred_speed_label = f"Pred Speed: {predicted_speed:.2f} m/s"#using this to display on the pygame display

                target_speed = raw_speed

        # PID throttle/brake control
        if target_speed is not None:

            current_speed = get_speed(vehicle)#get the vehicles current speed
            speed_error = target_speed - current_speed #get the overall difference

            now_t = time.time()
            dt = max(1e-3, now_t - prev_time) #calculate time difference

            if abs(speed_error) < IGNORE_ERROR: #ignore small errors
                throttle_cmd = 0.0
                brake_cmd = 0.0
            else:
                
                #PID integrator
                integral_err = clamp(integral_err + speed_error * dt, -INTEGRAL_CLAMP, INTEGRAL_CLAMP) #correction over time
                rate_of_change = (speed_error - prev_err) / dt #calculate rate of change

                pid_output = (
                    PROPORTIONAL_GAIN * speed_error
                    + INTEGRAL_GAIN * integral_err
                    + DERIVATIVE_GAIN * rate_of_change
                )#PID output formula calculation

                if pid_output >= 0:
                    # accelerating
                    throttle_cmd = clamp(pid_output, 0.0, MAX_ACCEL_THROTTLE)
                    brake_cmd = 0.0
                else:
                    # braking
                    throttle_cmd = 0.0
                    brake_cmd = clamp(BRAKE_GAIN_MULT * (-pid_output), 0.0, MAX_BRAKE)
                    if 0.0 < brake_cmd < MIN_BRAKE_TO_BITE:
                        brake_cmd = MIN_BRAKE_TO_BITE
                    integral_err = 0.0  # reset integral when we switch to braking to stop conflicting the brakes

            prev_err = speed_error
            prev_time = now_t

            control.throttle = throttle_cmd
            control.brake = brake_cmd
        else:#if there is no target speed (predicted speed, we dont move)
            control.throttle = 0.0
            control.brake = 0.0


        # if camera_np is not None:
        #     with torch.no_grad():
        #         img = image_transform(camera_np).unsqueeze(0).to(device)
        #         signal = torch.tensor([[turn_signal + 1]], dtype=torch.long).to(device)
        #         output = model(img, signal.squeeze(1))
        #         pred_class = output.argmax(dim=1).item()
        #         pred_label = CLASS_NAMES[pred_class]
        #         steering_value = steering_map[pred_class]
        #         if turn_signal == -1:
        #             if steering_value > -0.02:
        #                 steering_value = -0.02                  
        #         elif turn_signal == 1:
        #             if steering_value < 0.02:
        #                 steering_value = 0.02

        #         control.steer = steering_value
                

        vehicle.apply_control(control)

        # === Spectator follows vehicle ===
        car_transform = vehicle.get_transform()
        forward_vector = car_transform.get_forward_vector()
        cam_location = car_transform.location - forward_vector * 8 + carla.Location(z=3)
        cam_rotation = carla.Rotation(pitch=-10, yaw=car_transform.rotation.yaw)
        spectator.set_transform(carla.Transform(cam_location, cam_rotation))

        # === Display Feed + Prediction ===
        if camera_image:
            screen.blit(camera_image, (0, 0))
            font = pygame.font.SysFont(None, 30)
            label_surface = font.render(f"Predicted: {pred_label}", True, (255, 255, 0))
            screen.blit(label_surface, (20, 20))

            if predicted_speed is not None:
                hud_pred = font.render(pred_speed_label, True, (0, 255, 0))
                screen.blit(hud_pred, (20, 50))

            current_speed_mps = get_speed(vehicle)
            hud_curr = font.render(f"Current: {current_speed_mps:.2f} m/s", True, (0, 200, 255))
            screen.blit(hud_curr, (20, 80))

            pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT or keys[pygame.K_ESCAPE]:
                raise KeyboardInterrupt

except KeyboardInterrupt:
    print("Exiting and cleaning up...")

finally:
    camera.stop()
    vehicle.destroy()
    camera.destroy()
    pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\deyru\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\deyru\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model-controlled driving started.
Q/E to turn on left/right signal, R to cancel.
ESC or close window to exit.
